In [ ]:
# !pip install datasets
!pip install tensorflow-gpu==2.9.0

In [ ]:
size = 8736

In [ ]:
import pickle
features = pickle.load(open("/content/drive/MyDrive/MajorProject/features.pkl", "rb"))
# features = list(features.values())
features = features[:size]
print("Completed!")
print(len(features))

In [ ]:
import random
import numpy as np
# from datasets import load_dataset
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import re
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import Input
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.layers import Reshape, concatenate, add
from keras.layers import Dense, Dropout
from keras.utils.np_utils import to_categorical
from tqdm import tqdm
from tensorflow.keras.applications import DenseNet201
from keras.models import Sequential, Model
from keras.layers import Embedding, LSTM, GRU
import tensorflow

In [ ]:
# plt.rcParams['font.size'] = 12
# sns.set_style("dark")
# warnings.filterwarnings('ignore')

In [ ]:
def text_preprocessing(dataCap):
    for index in range(len(dataCap)):
        caption = dataCap[index]
        caption = caption.lower()
        # caption = re.sub(re.compile('[^A-Za-z]'), '', caption)
        # caption = " ".join(caption.split())
        # caption = re.sub(r"(?<![a-zA-Z])[a-zA-Z](?![a-zA-Z])", "", caption)
        caption = "startseq " + caption + " endseq"
        dataCap[index] = caption
    return dataCap

# LOADING THE DATASET
# dataset = load_dataset("arampacha/rsicd")
# print(dataset)

# # ARRANGING THE DATA
# print("STARTING ARRANGEMENT...")
# dataImage = dataset["train"]["image"] + dataset["test"]["image"] + dataset["valid"]["image"]
# dataCap = dataset["train"]["captions"] + dataset["test"]["captions"] + dataset["valid"]["captions"]
# dataFilename = dataset["train"]["filename"] + dataset["test"]["filename"] + dataset["valid"]["filename"]

dataImage = pickle.load(open("/content/drive/MyDrive/MajorProject/dataImages.pkl", "rb"))
dataCap = pickle.load(open("/content/drive/MyDrive/MajorProject/dataCaptions.pkl", "rb"))
dataFilename = pickle.load(open("/content/drive/MyDrive/MajorProject/dataFilename.pkl", "rb"))
dataImage = dataImage[:size]
dataCap = dataCap[:size]
dataFilename = dataFilename[:size]

for idx in range(len(dataCap)):
    dataCap[idx] = dataCap[idx]
print("Images length: ", len(dataImage))
print("Captions length: ", len(dataCap))
dataCap = text_preprocessing(dataCap)
print('\n'.join(map(str, dataCap[:5])))

# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(dataCap)
# vocab_size = len(tokenizer.word_index) + 1
# max_length = max(len(caption.split()) for caption in dataCap)

# ndataImages = len(dataImage)
# split_index = round(0.70 * ndataImages)
# train = dataImage[:split_index]
# test = dataImage[split_index:]
# trainCaps = dataCap[:split_index]
# testCaps = dataCap[split_index:]


# model = DenseNet201()
# fe = Model(inputs=model.input, outputs=model.layers[-2].output)
# features = {}
# for imageO in tqdm(dataImage):
#     img = np.array(imageO)
#     img = img.reshape((224,224,3))
#     img = img / 255.0
#     img = np.expand_dims(img, axis=0)
#     feature = fe.predict(img, verbose=0)
#     idx = dataImage.index(imageO)
#     filename = dataFilename[idx]
#     features[filename] = feature

In [ ]:
# pickle.dump(dataImage, open("/content/drive/MyDrive/MajorProject", "wb"))

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(dataCap)
vocab_size = len(tokenizer.word_index) + 1
max_length = max(len(caption.split()) for caption in dataCap)

ndataImages = len(dataImage)
split_index = round(0.85 * ndataImages)
train = dataImage[:split_index]
test = dataImage[split_index:]
trainCaps = dataCap[:split_index]
testCaps = dataCap[split_index:]


model = DenseNet201()
fe = Model(inputs=model.input, outputs=model.layers[-2].output)

In [ ]:
class CustomDataGenerator(Sequence):
    def __init__(self, X, y, batch_size, tokenizer, dataFilename,
                  vocab_size, max_length, features, shuffle=False):

        self.X = X.copy()
        self.y = y.copy()
        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.vocab_size = vocab_size
        self.max_length = max_length
        self.features = features
        self.dataFilename = dataFilename
        self.shuffle = shuffle
        self.n = len(self.X)

    def on_epoch_end(self):
        if self.shuffle:
            zipped = list(zip(self.X, self.y))
            random.shuffle(zipped)
            self.X, self.y = zip(*zipped)

    def __len__(self):
        return self.n // self.batch_size

    def __getitem__(self, index):

        X1, X2, y = self.__get_data(index)
        return (X1, X2), y

    def __get_data(self, index):

        X1, X2, y = list(), list(), list()
        start_index = index * self.batch_size
        end_index = start_index + self.batch_size

        images = self.X[start_index:end_index]
        captions = self.y[start_index:end_index]
        length = len(images)
        for idx in range(length):
            image = images[idx]
            # fn= dataFilename[idx]
            feature = self.features[idx][0]
            caption = captions[idx]
            # captions = self.y[start_index:end_index]
            # for caption in captions:
            seq = self.tokenizer.texts_to_sequences([caption])[0]

            for i in range(1, len(seq)):
                in_seq, out_seq = seq[:i], seq[i]
                in_seq = pad_sequences([in_seq], maxlen=self.max_length)[0]
                out_seq = to_categorical([out_seq], num_classes=self.vocab_size)[0]
                X1.append(feature)
                X2.append(in_seq)
                y.append(out_seq)

        X1, X2, y = np.array(X1), np.array(X2), np.array(y)

        return X1, X2, y

In [ ]:
# def text_preprocessing(dataCap):
#     for index in range(len(dataCap)):
#         caption = dataCap[index]
#         caption = caption.lower()
#         # caption = re.sub(re.compile('[^A-Za-z]'), '', caption)
#         # caption = " ".join(caption.split())
#         # caption = re.sub(r"(?<![a-zA-Z])[a-zA-Z](?![a-zA-Z])", "", caption)
#         caption = "startseq " + caption + " endseq"
#         dataCap[index] = caption
#     return dataCap

# # LOADING THE DATASET
# dataset = load_dataset("arampacha/rsicd")
# print(dataset)

# # ARRANGING THE DATA
# print("STARTING ARRANGEMENT...")
# dataImage = dataset["train"]["image"] + dataset["test"]["image"] + dataset["valid"]["image"]
# dataCap = dataset["train"]["captions"] + dataset["test"]["captions"] + dataset["valid"]["captions"]
# dataFilename = dataset["train"]["filename"] + dataset["test"]["filename"] + dataset["valid"]["filename"]
# for idx in range(len(dataCap)):
#     dataCap[idx] = dataCap[idx][1]
# print("Images length: ", len(dataImage))
# print("Captions length: ", len(dataCap))
# dataCap = text_preprocessing(dataCap)
# print('\n'.join(map(str, dataCap[:5])))

# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(dataCap)
# vocab_size = len(tokenizer.word_index) + 1
# max_length = max(len(caption.split()) for caption in dataCap)

# ndataImages = len(dataImage)
# split_index = round(0.70 * ndataImages)
# train = dataImage[:split_index]
# test = dataImage[split_index:]
# trainCaps = dataCap[:split_index]
# testCaps = dataCap[split_index:]


# model = DenseNet201()
# fe = Model(inputs=model.input, outputs=model.layers[-2].output)
# features = {}
# for imageO in tqdm(dataImage):
#     img = np.array(imageO)
#     img = img.reshape((224,224,3))
#     img = img / 255.0
#     img = np.expand_dims(img, axis=0)
#     feature = fe.predict(img, verbose=0)
#     idx = dataImage.index(imageO)
#     filename = dataFilename[idx]
#     features[filename] = feature

In [ ]:
# import pickle
# with open('/content/drive/MyDrive/MajorProject/features.pkl', 'wb') as f:
#     pickle.dump(features, f)

In [ ]:
from tensorflow.keras.optimizers import Adam

input1 = Input(shape=(1920,))
input2 = Input(shape=(max_length,))
img_features = Dense(1024, activation='relu')(input1)
img_features_reshaped = Reshape((1, 1024), input_shape=(1024,))(img_features)
sentence_features = Embedding(vocab_size, 1024, mask_zero=False)(input2)
merged = concatenate([img_features_reshaped, sentence_features], axis=1)
sentence_features = LSTM(1024)(merged)
x = Dropout(0.25)(sentence_features)
x = add([x, img_features])
x = Dense(1024, activation='relu')(x)
x = Dropout(0.25)(x)
output = Dense(vocab_size, activation='softmax')(x)

caption_model = Model(inputs=[input1, input2], outputs=output)
model_optimizer = Adam(learning_rate=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
caption_model.compile(loss='categorical_crossentropy', optimizer=model_optimizer,metrics = ['accuracy'])
caption_model.summary()

train_generator = CustomDataGenerator(train, trainCaps, batch_size=64, tokenizer=tokenizer,
                                      dataFilename=dataFilename[:split_index],
                                      vocab_size=vocab_size, max_length=max_length, features=features)

validation_generator = CustomDataGenerator(test, testCaps, batch_size=64, tokenizer=tokenizer,
                                            dataFilename=dataFilename[split_index:],
                                            vocab_size=vocab_size, max_length=max_length, features=features)
print(train_generator)
print(validation_generator)

In [ ]:
# model_name = "ImageCaptioningmodel_100_V1.h5"
# checkpoint = ModelCheckpoint(model_name,
#                               monitor="val_loss",
#                               mode="min",
#                               save_best_only=True,
#                               verbose=1)
# earlystopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5,
#                               verbose=1, restore_best_weights=True)

# learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
#                                             patience=3,
#                                             verbose=1,
#                                             factor=0.2)

history = caption_model.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator)
caption_model.save('/content/drive/MyDrive/MajorProject/ImageCaptioningmodel_100_V1.h5')
print(history.history.keys())

In [ ]:
test = pickle.load(open("/content/drive/MyDrive/MajorProject/dataImages.pkl", "rb"))
test = test[size:]
print(len(test))

In [ ]:
from tensorflow.keras.models import load_model
# img_path = "/content/drive/MyDrive/MajorProject/church_015.jpg"
def idx_to_word(integer,tokenizer):
    
    for word, index in tokenizer.word_index.items():
        if index==integer:
            return word
    return None



def predict_caption(model, img, tokenizer, max_length, features):
    
    feature = fe.predict(img, verbose=0)
    # for fv in features.values():
    #   if feature == fv:
    #     break
    #   res = feature0

    in_text = "startseq"
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], max_length)

        y_pred = model.predict([feature,sequence])
        y_pred = np.argmax(y_pred)
        
        word = idx_to_word(y_pred, tokenizer)
        
        if word is None:
            break
            
        in_text+= " " + word
        
        if word == 'endseq':
            break
            
    return in_text 

# img = load_img(img_path)

# # img = img_to_array(img)
# # img = np.array(img)
# img = img_to_array(img)

# img = load_img(img_path)
img = np.array(test[2000])
# img = img_to_array(img)
img = cv2.resize(img,(224,224))
img = np.expand_dims(img, axis=0)

saved_model = load_model('/content/drive/MyDrive/MajorProject/ImageCaptioningmodel_100_V1.h5')
with open('/content/drive/MyDrive/MajorProject/features.pkl', 'rb') as f:
    featuresS = pickle.load(f)
caption = predict_caption(saved_model, img, tokenizer, max_length, featuresS)
caption = caption.replace("startseq", "")
caption = caption.replace("endseq","")
print(caption)

In [ ]:
import matplotlib.pyplot as plt
img = np.array(test[2000])
plt.imshow(img)
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig("/content/drive/MyDrive/MajorProject/ImageCaptioningmodel_100_V1.png")
plt.show()